In [1]:
!rm -rf /kaggle/working/multimodal-eq-sizing
!git clone https://github.com/brianrp09232000/multimodal-eq-sizing.git /kaggle/working/multimodal-eq-sizing
!pip install -r /kaggle/working/multimodal-eq-sizing/requirements.txt

Cloning into '/kaggle/working/multimodal-eq-sizing'...
remote: Enumerating objects: 444, done.
remote: Counting objects: 100% (64/64), done.
remote: Compressing objects: 100% (34/34), done.
remote: Total 444 (delta 45), reused 30 (delta 30), pack-reused 380 (from 3)
Receiving objects: 100% (444/444), 162.78 KiB | 5.61 MiB/s, done.
Resolving deltas: 100% (255/255), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.4/123.4 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.6/52.6 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.9/259.9 kB 12.6 MB/s eta 0:00:00
  Created wheel for typing: filename=typing-3.7.4.3-py3-none-any.whl size=26304 sha256=823f96297c5121e9e03179ea26c06cbbe3d1a706e3288c072c54a78662fba9cf
  Stored in directory: /root/.cache

In [2]:
import sys
import pathlib
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

In [3]:
np.seterr(invalid="ignore")

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

In [4]:
repo_root = pathlib.Path("/kaggle/working/multimodal-eq-sizing")
sys.path.append(str(repo_root))

In [5]:
from src.data.loaders import (
    get_tickers_history,
    get_return_data,
    get_excess_return,
    get_vix_data,
    get_spread_z,
    get_sector_map,
    get_adv_dollar
)

# Find Tickers with the Most Headlines

In [6]:
from src.data.universe import tickers_with_most_headlines

In [7]:
news_df = pd.read_csv("/kaggle/input/financial-news/All_external.csv") 

/tmp/ipykernel_13/138064575.py:1: DtypeWarning: Columns (2,4,5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  news_df = pd.read_csv("/kaggle/input/financial-news/All_external.csv")


In [8]:
def count_headlines_per_ticker(news_df, start=None, end=None):
    """Counts the number of headlines for each ticker symbol 
    Input: news_df pandas dataframe with ticker column for ticker symbols
    Output: pandas dataframe containing two columns: ticker names and the 
                number of headlines for the ticker"""
    
    #check columns in dataframe
    columns = list(news_df.columns)
    if (('date' not in columns) and ('Date' not in columns)) or (('ticker' not in columns) and ('Stock_symbol' not in columns)):
        print('input dataframe does not have both ticker and date columns')
        return pd.DataFrame()
    
    #find column names
    date_col = 'date' if 'date' in columns else 'Date'
    ticker_col = 'ticker' if 'ticker' in columns else 'Stock_symbol'

    #filter dates
    if start is not None: 
        start_filter = news_df[date_col] >= str(start)
        news_df = news_df[start_filter]
    if end is not None: 
        end_filter = news_df[date_col] <= str(end)
        news_df = news_df[end_filter]
    
    # Count occurrences in a specific column
    headline_counts = news_df[ticker_col].value_counts()
    df = headline_counts.to_frame(name='count')
    df['ticker'] = list(df.index)
    df = df.reset_index(drop=True)
    
    return df[['ticker','count']]



def tickers_with_most_headlines(news_df, start=None, end=None, n=200):
    """Finds the tickers with the most headlines 
    Input: news_df pandas dataframe with ticker column for ticker symbols
            optional: n interger, number of top tickers to return
    Output: list containing the number of headlines per ticker
                for the tickers with the most headlines"""
    
    #count headlines for each ticker
    df = count_headlines_per_ticker(news_df, start, end)

    #limit dataframe to n tickers
    df = df.sort_values(['count'], ascending=False)
    df = df[:n]
    df.reset_index(drop=True, inplace=True)
    
    return df

In [9]:
start = pd.Timestamp('2010-01-04 05:00:00+0000', tz='UTC')
end   = pd.Timestamp('2018-12-28 05:00:00+0000', tz='UTC')
tickers = tickers_with_most_headlines(news_df, str(start), str(end), 300)

# Get Price Data

In [10]:
#get yfinance ticker history for all tickers in tickers df
#yfinance will produce the "possibly delisted" message for tickers without information
df = get_tickers_history(list(tickers['ticker']), start=start, end=end)

$DISH: possibly delisted; no timezone found
HTTP Error 404: {"quoteSummary":{"result":null,"error":{"code":"Not Found","description":"Quote not found for symbol: YHOO"}}}
$YHOO: possibly delisted; no timezone found
$MYL: possibly delisted; no timezone found
$GMCR: possibly delisted; no price data found  (1d 2010-01-04 05:00:00+00:00 -> 2018-12-29 05:00:00+00:00)
$JWN: possibly delisted; no timezone found
$VMW: possibly delisted; no timezone found
$WFM: possibly delisted; no price data found  (1d 2010-01-04 05:00:00+00:00 -> 2018-12-29 05:00:00+00:00)
$BHI: possibly delisted; no price data found  (1d 2010-01-04 05:00:00+00:00 -> 2018-12-29 05:00:00+00:00)
$FCAU: possibly delisted; no timezone found
$RAD: possibly delisted; no timezone found
$DOW: possibly delisted; no price data found  (1d 2010-01-04 05:00:00+00:00 -> 2018-12-29 05:00:00+00:00) (Yahoo error = "Data doesn't exist for startDate = 1262581200, endDate = 1546059600")
$TIF: possibly delisted; no timezone found
$ABX: possibly 

In [11]:
#remove unnecessary columns
df = df.drop(['Capital Gains','Adj Close'], axis=1)

In [12]:
#limit df to only 200 tickers and tickers with data
keep_tickers = list(df['ticker'].drop_duplicates()[:200])
df = df[df['ticker'].isin(keep_tickers)]
tickers = tickers[tickers['ticker'].isin(keep_tickers)]

In [13]:
def get_date_range(df: pd.DataFrame) -> tuple:
    grouped_by_date = df.groupby(["ticker"]).agg(['min', 'max', 'count'])["Date"]
    start = grouped_by_date["min"].min()
    end = grouped_by_date["max"].max()
    return start, end

In [14]:
#df = get_return_data("/kaggle/input/news-trading/return_data.csv")
#start, end = get_date_range(df)

# Add excess return

In [15]:
def add_excess_return(df, start, end):
    excess_return_df = get_excess_return(df, start, end)
    df = df.merge(excess_return_df, on=["ticker", "Date"], how="left")
    return df

In [16]:
df = add_excess_return(df, start, end)

# Add market regime VIX z-score

In [17]:
def add_vix_z(df, start, end):
    vix_z_df = get_vix_data(start, end)
    format_str = "%Y-%m-%d"
    vix_z_df["Date"] = vix_z_df["Date"].dt.strftime(format_str)
    df["Date"] = df["Date"].dt.strftime(format_str) 
    df = df.merge(vix_z_df, on=["Date"], how="left")
    df['Date'] = pd.to_datetime(df['Date'], utc=True)
    return df

In [18]:
df = add_vix_z(df, start, end)

Yay!🥳


# Add spread z-score

In [19]:
def add_spread_z(existing_df: pd.DataFrame, buffer_days=380) -> pd.DataFrame:
    """
    Use existing OHLCV df, pull buffered history, compute young-safe spread_z on the combined
    Then merge back only the target window rows to prevent nulls.
    """
    df = existing_df.copy()
    start, end = df["Date"].min(), df["Date"].max()

    tickers = sorted(df['ticker'].unique())
    fetch_start = start - timedelta(days=buffer_days)
    fetch_end   = end

    # You already have get_tickers_history(tickers, start, end)
    hist = get_tickers_history(tickers, fetch_start, fetch_end)
    hist["Date"] = pd.to_datetime(hist["Date"], utc=True)

    # Combine buffer + existing; keep existing rows on overlap
    combined = pd.concat([hist, df], ignore_index=True)
    combined = combined.sort_values(['ticker', "Date"])
    combined = combined.drop_duplicates(subset=['ticker', "Date"], keep="last")

    # Compute young-safe spread_z on the full combined range
    combined = get_spread_z(combined)

    # Merge only computed columns back to target window
    cols_to_merge = ['ticker', 'Date', "spread_z"]
    out = df.merge(combined[cols_to_merge], on=['ticker', 'Date'], how="left")

    # Final minimal, causal clean-up to guarantee NON-NULL spread_z in target window:
    # 1) per-ticker forward-fill (past only), 2) same-day cross-section median, 3) final 0
    out["spread_z"] = (
        out.groupby('ticker')["spread_z"].ffill()
           .fillna(out.groupby('Date')["spread_z"].transform("median"))
           .fillna(0.0)
    ).clip(-3, 3)

    return out

In [20]:
df = add_spread_z(df)

# Add sector

In [21]:
def add_sector(df):
    tickers = df["ticker"].unique()
    sector_map = get_sector_map(tickers)
    df = df.join(sector_map, on="ticker")
    return df

In [22]:
df = add_sector(df)

# Add dollar-volume ADV 

In [23]:
def add_adv_dollar(df):
    adv_df = get_adv_dollar(df)
    
    df = df.merge(
        adv_df,
        on=["Date", "ticker"],
        how="left",
    )
    return df

In [24]:
df = add_adv_dollar(df)

# Final Complete Dataset

In [25]:
df.to_csv('final_dataset.csv', index=False)

In [26]:
tickers.to_csv('top_tickers.csv', index=False)

In [27]:
get_return_data("/kaggle/working/final_dataset.csv")

/kaggle/working/multimodal-eq-sizing/src/data/loaders.py:9: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df_ticker_return = pd.read_csv(path)


,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,ticker,o2c_return,excess_return,VIX_Close,VIX_z,spread_z,sector,dollar_volume,adv_dollar
0,2010-01-04 00:00:00+00:00,7.590514,7.634881,7.557238,7.572027,12718800.0,0.0,0.0,KR,-0.002436,-0.010979,20.040001,-1.249591,1.316371,Consumer Defensive,9.630710e+07,NaN
1,2010-01-05 00:00:00+00:00,7.579421,7.597907,7.457411,7.483292,19904000.0,0.0,0.0,KR,-0.012683,-0.015950,19.350000,-1.314181,0.482728,Consumer Defensive,1.489474e+08,NaN
2,2010-01-06 00:00:00+00:00,7.483294,7.520267,7.398256,7.486991,25134000.0,0.0,0.0,KR,0.000494,-0.001180,19.160000,-1.323599,0.342750,Consumer Defensive,1.881780e+08,NaN
3,2010-01-07 00:00:00+00:00,7.486990,7.568330,7.420439,7.560936,21133400.0,0.0,0.0,KR,0.009877,0.003797,19.059999,-1.324017,0.740290,Consumer Defensive,1.597883e+08,NaN
4,2010-01-08 00:00:00+00:00,7.564630,7.564630,7.438923,7.490685,20974200.0,0.0,0.0,KR,-0.009775,-0.015746,18.129999,-1.413993,1.029473,Consumer Defensive,1.571111e+08,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
436222,2018-12-21 00:00:00+00:00,11.700000,11.960000,11.630000,11.690000,9557300.0,0.0,0.0,CNX,-0.000855,0.023625,30.110001,2.909232,0.028351,Energy,1.117248e+08,3.813725e+07
436223,2018-12-24 00:00:00+00:00,11.500000,11.610000,10.860000,10.920000,2357100.0,0.0,0.0,CNX,-0.050435,-0.030773,36.070000,3.000000,-3.000000,Energy,2.573953e+07,3.861721e+07
436224,2018-12-26 00:00:00+00:00,11.000000,12.000000,10.780000,12.000000,3368000.0,0.0,0.0,CNX,0.090909,0.047641,30.410000,2.812017,-1.974304,Energy,4.041600e+07,3.915679e+07
436225,2018-12-27 00:00:00+00:00,11.710000,12.100000,11.560000,12.090000,3032000.0,0.0,0.0,CNX,0.032451,0.009777,29.959999,2.675121,-2.893259,Energy,3.665688e+07,3.932039e+07


In [28]:
!rm -rf /kaggle/working/multimodal-eq-sizing